# Wstęp do Uczenia Maszynowego - Lab 5

Wszystkie dzisiejsze metody można znaleźć dokładnie (i przystępnie!) omówione na StatQuest: https://www.youtube.com/user/joshstarmer

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
np.random.seed = 42


In [3]:

COLOR = 'white'
matplotlib.rcParams['text.color'] = COLOR
matplotlib.rcParams['axes.labelcolor'] = COLOR
matplotlib.rcParams['xtick.color'] = COLOR
matplotlib.rcParams['ytick.color'] = COLOR
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['font.size'] = 14


In [4]:
data = pd.read_csv('heart.csv')
data.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


In [5]:
y = np.array(data['chd'])
X = data.drop(['chd'],axis=1)

In [6]:
map_dict = {'Present': 1, 'Absent':0}
X['famhist'] = X['famhist'].map(map_dict)
X.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age
0,160,12.00,5.73,23.11,1,49,25.30,97.20,52
1,144,0.01,4.41,28.61,0,55,28.87,2.06,63
2,118,0.08,3.48,32.28,1,52,29.14,3.81,46
3,170,7.50,6.41,38.03,1,51,31.99,24.26,58
4,134,13.60,3.50,27.78,1,60,25.99,57.34,49


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

## Ensemble Methods - Komitety
Często kilka osób, które myślą nad daną sprawą, potrafi dać lepszą odpowiedź niż jedna osoba. Nawet jeśli żadna z osób nie jest ekspertem.  
To podejście zadziała, gdy błędny popełniane przez różne modele są od siebie niezależne (w miarę). 

  
Na potrzeby stosowania różnych metod Ensemble Learningu załadujemy sobie już 3 modele z których będziemy potem korzystać.

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

model1 = DecisionTreeClassifier(max_depth = 10, random_state=1)
model2 = KNeighborsClassifier()
model3 = LogisticRegression(random_state=1, max_iter=1000)
estimators=[('DecisionTree', model1), ('KNN', model2), ('LR', model3)]

In [9]:
# czy coś tu nie gra?

# @conclusions
# nie wiemy do jakich danych
# max_depth dla tree nie jest zdefiniowane domyślnie, więc na 90% overfitting

In [12]:
model_lr = LogisticRegression(random_state=1, max_iter=1000)
model_lr.fit(X_train, y_train)
model_lr.score(X_test, y_test)

0.7634408602150538

In [22]:
model_lr.predict(X_test)

array([0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0])

In [24]:
model_lr.predict_proba(X_test)[:10]

array([[0.7012877 , 0.2987123 ],
       [0.75003096, 0.24996904],
       [0.14134935, 0.85865065],
       [0.49614457, 0.50385543],
       [0.69339818, 0.30660182],
       [0.60127493, 0.39872507],
       [0.76427284, 0.23572716],
       [0.54772816, 0.45227184],
       [0.4603363 , 0.5396637 ],
       [0.40133013, 0.59866987]])

### Max Voting
lub Hard Voting - głosowanie większościowe 

In [10]:
from sklearn.ensemble import VotingClassifier

In [14]:
model_hard = VotingClassifier(estimators=estimators, voting='hard')
model_hard.fit(X_train,y_train)

y_hat = model_hard.predict(X_test)
print('accuracy: ', accuracy_score(y_test, y_hat))

# model też ma metodę ewaluacji i jest to też accuracy w przypadku klasyfikacji
print('model.score: ', model_hard.score(X_test,y_test))

accuracy:  0.7311827956989247
model.score:  0.7311827956989247


### Averaging
Soft Voting. Nie patrzymy na liczbę głosów, ale na "pewność". Patrzymy na ile pewny jest klasyfikator, że rekord należy do klasy 1.   
c1 - 90%, c2 - 49%, c3 - 49% 
* hard voting: klasa 0 (1 głos na tak, 2 głosy na nie)
* soft voting: klasa 1 ( (90 + 49 + 49)/3 $\approx$ 63% > 50%)


In [20]:
model_soft = VotingClassifier(estimators=estimators, voting='soft')
model_soft.fit(X_train, y_train)

y_hat = model_soft.predict(X_test)
accuracy_score(y_test, y_hat)

0.6344086021505376

# Weights
Nie musimy zawsze uważać, że każdy klasyfikator ma tyle samo do powiedzenia.

In [21]:
model_soft = VotingClassifier(estimators=estimators, voting='soft', weights=[0.05, 0.05, 0.90])
model_soft.fit(X_train, y_train)

y_hat = model_soft.predict(X_test)
accuracy_score(y_test, y_hat)

0.7526881720430108

### Stacking
<div>
<img src="https://miro.medium.com/max/700/1*RP0pkQEOSrw9_EjFu4w3gg.png" />
</div>
sos: https://miro.medium.com/max/700/1*RP0pkQEOSrw9_EjFu4w3gg.png

Bierzemy kilka różnych modeli (base) i jeden meta-model. Meta-model uczy się przewidywać wynik na podstawie predykcji z base.

Czy można stackować kilka takich samych modeli?  
 
@conclusions  
No raczej nie, bo wyniki modeli w base powinny być od siebie niezależne.  
Stackowanie jeden na drugim tych samych modeli -> check

Zasada kciuka: ostatni model jest raczej prosty (regresja liniowa/logistyczna).  
Więcej (np. o trenowaniu) tutaj: https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/

In [25]:
from sklearn.ensemble import StackingClassifier

In [28]:
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X_train, y_train).score(X_test, y_test)

0.7849462365591398

In [33]:
clf2 = StackingClassifier(estimators=[('lr', model3)], final_estimator=LogisticRegression())
clf2.fit(X_train, y_train).score(X_test, y_test) - model_lr.score(X_test, y_test)

0.0

### Bagging (Bootstrap Aggregating)
Bootstrap - to technika próbkowania, w której tworzymy podzbiory (próby) obserwacji z oryginalnego datasetu, **ze zwracaniem**. Rozmiar podzbiorów jest taki sam jak rozmiar oryginalnego datasetu.

1. Losujemy N **bootstrapowych** prób ze zbioru treningowego
2. Trenujemy niezależnie N "słabych" klasyfikatorów
3. Składamy wyniki "słabych" modeli 
    - **Klasyfikacja:** reguła większościowa / uśrednione prawdopodobieństwo
    - **Regresja:** Uśrednione wartości

In [34]:
from sklearn.ensemble import BaggingClassifier

In [35]:
clf = BaggingClassifier(base_estimator=model1,
                        n_estimators=10, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.6666666666666666

## Random Forest
Najbardziej popularny algorytm Baggingowy. Wiele małych drzew.

Przypomnijmy
#### Zalety drzew
* interpretowalność
* prostota + wizualizacja
* nie trzeba normalizować danych \:) i outliery się ich nie tykają
* mają fajne metody do wykrywania ważnych cech

#### Wady drzew
* łatwo o overfitting
* drzewo długo rośnie
* starych drzew się nie przesadza - nie da się dotrenować algorytmu po jakimś czasie, trzeba zasadzić nowe 
* jest duża losowość - na tych samych danych algorytm może dawać bardzo różne wyniki

Zastanówmy się nad zaletami RF  
@conclusions  

#### Zalety:
* trudniej o overfitting
* mają fajne metody do wykrywania ważnych cech
* nie trzeba normalizować danych \:) i outliery się ich nie tykają
* mniejsza losowość

#### Wady:
* las rośnie dłużej niż drzewo
* lasu też się nie przesadza
* tracimy interpretowalność


In [40]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=1000, # Ilość słabych estymatorów
                                  max_depth=5, # Maksymalna wysokość drzewa w słabym estymatorze
                                  min_samples_split = 2, # Minimalna ilość obserwacji wymagana do podziału węzła
                                  max_features = 5, # Maksymalna ilość zmiennych brana pod uwagę przy podziale węzła
                                  random_state=0,
                                  n_jobs = -1)
model_rf.fit(X_train, y_train)
model_rf.score(X_test,y_test)

0.7634408602150538

### Boosting
Boosting działa podobnie jak Bagging z jedną różnicą. Każda kolejna próba bootstrap jest tworzona w taki sposób,  
że losuje z większym prawdopodobieństwiem obserwacje **źle sklasyfikowane**.  
W skrócie: Boosting uczy się na błędach, które popełnił w poprzednich iteracjach.

#### AdaBoost
Najprostsza metoda boostingowa.  
Budujemy wiele bardzo niskich drzew, np. wysokości 1.  
Modele, które sobie radzą lepiej, mają większe prawo głosu.

In [49]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(random_state=1, n_estimators=20, learning_rate=0.5)
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.7741935483870968

#### Gradient Boosting
Zaczynamy od pojedynczej predykcji (np. średniej) i liczymy różnicę, tzw. residuum.  
Następnie każdy model (niskie drzewo, niekoniecznie wysokości 1) próbuje przewidzieć residuum.  
W GB każdy model ma taki sam głos.

In [50]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=1,
                                  learning_rate=0.01)
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.7204301075268817

#### XGBoost
Zaawansowana implementacja Gradient Boostingu

In [52]:
from xgboost import XGBClassifier # Inna paczka niż sklearn!

In [53]:
model=XGBClassifier(random_state=1,
                    learning_rate=0.01, # Szybkość "uczenia" się
                    booster='gbtree', # Jaki model wykorzystujemy (drzewo - gbtree, liniowe - gblinear)
                    nround = 100, # Ilość itereacji boosingowych
                    max_depth=4 # Maksymalna głębokość drzewa 
                    )
model.fit(X_train, y_train)
model.score(X_test,y_test)

[17:18:01] WARNING: ../src/learner.cc:541: 
Parameters: { nround } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:18:01] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/igro_m23/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.7526881720430108

### Wszystkie dzisiejsze metody można znaleźć dokładnie (i przystępnie!) omówione na StatQuest: https://www.youtube.com/user/joshstarmer

https://mateuszgrzyb.pl/wybor-odpowiedniego-algorytmu-czesc-2-algorytmy-klasyfikacyjne/